This notebook contains exploratory data analysis on FAERS using the api.

Author - Akshay Chougule<br>
Created on - 5th November 2020<br>
Last Updated - 6th November 2020<br>
<br>
[Reference](https://open.fda.gov/apis/drug/event/how-to-use-the-endpoint/)

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import json
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('max_colwidth', 800)

## bevacizumab

### case-level query

In [14]:
# to get individual case records
query_url = 'https://api.fda.gov/drug/event.json?search=bevacizumab&limit=1000&skip=0'
# We can keep increasing skip by the limit number in the subsquent calls to get the next batch of the data
r = requests.get(query_url)
j = json.loads(r.content)

In [28]:
len(j['results'])

1000

In [29]:
j.keys()

dict_keys(['meta', 'results'])

In [30]:
j['meta']

{'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
 'terms': 'https://open.fda.gov/terms/',
 'license': 'https://open.fda.gov/license/',
 'last_updated': '2020-11-01',
 'results': {'skip': 0, 'limit': 1000, 'total': 77627}}

In [31]:
len(j['results'])

1000

In [32]:
j['results'][0].keys()

dict_keys(['reporttype', 'safetyreportversion', 'receivedateformat', 'primarysource', 'transmissiondateformat', 'fulfillexpeditecriteria', 'safetyreportid', 'patient', 'serious', 'reportduplicate', 'receiptdate', 'receiptdateformat', 'receiver', 'seriousnessdeath', 'companynumb', 'occurcountry', 'duplicate', 'sender', 'receivedate', 'transmissiondate', 'primarysourcecountry'])

In [33]:
j['results'][0]['patient'].keys()

dict_keys(['reaction', 'patientsex', 'drug'])

In [25]:
len(j['results'][0]['patient'])

3

In [26]:
j['results'][0]['patient']

{'reaction': [{'reactionmeddraversionpt': '17.0',
   'reactionmeddrapt': 'Death',
   'reactionoutcome': '5'}],
 'patientsex': '1',
 'drug': [{'drugindication': 'COLORECTAL CANCER METASTATIC',
   'drugadministrationroute': '042',
   'drugcharacterization': '1',
   'openfda': {'product_ndc': ['50242-060', '50242-061'],
    'spl_id': ['bcde0461-1e76-4e4c-a643-1543436d2cda'],
    'substance_name': ['BEVACIZUMAB'],
    'product_type': ['HUMAN PRESCRIPTION DRUG'],
    'route': ['INTRAVENOUS'],
    'package_ndc': ['50242-060-01',
     '50242-060-10',
     '50242-061-01',
     '50242-061-10'],
    'generic_name': ['BEVACIZUMAB'],
    'spl_set_id': ['939b5d1f-9fb2-4499-80ef-0607aa6b114e'],
    'brand_name': ['AVASTIN'],
    'application_number': ['BLA125085'],
    'manufacturer_name': ['Genentech, Inc.'],
    'unii': ['2S9ZZM9Q9V']},
   'drugauthorizationnumb': '125085',
   'medicinalproduct': 'AVASTIN'}]}

### aggregated query

In [34]:
# to get aggregated data
query_url = 'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000&skip=0'
# We can keep increasing skip by the limit number in the subsquent calls to get the next batch of the data
r = requests.get(query_url)
j = json.loads(r.content)

In [35]:
j

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2020-11-01'},
 'results': [{'term': 'DEATH', 'count': 420},
  {'term': 'DISEASE PROGRESSION', 'count': 256},
  {'term': 'DIARRHOEA', 'count': 106},
  {'term': 'NAUSEA', 'count': 98},
  {'term': 'VOMITING', 'count': 92},
  {'term': 'FATIGUE', 'count': 72},
  {'term': 'OFF LABEL USE', 'count': 71},
  {'term': 'PYREXIA', 'count': 66},
  {'term': 'GENERAL PHYSICAL HEALTH DETERIORATION', 'count': 63},
  {'term': 'ABDOMINAL PAIN', 'count': 60},
  {'term': 'ASTHENIA', 'count': 58},
  {'term': 'PNEUMONIA', 'count': 53},
  {'term': 'DYSPNOEA', 'count': 51},
  {'term': 'CONSTIPATION', 'count': 49},
  {'t

In [39]:
agg_df = pd.DataFrame(j['results'])
agg_df.head()

,term,count
0,DEATH,420
1,DISEASE PROGRESSION,256
2,DIARRHOEA,106
3,NAUSEA,98
4,VOMITING,92


In [44]:
continue_flag = 1
limit=1000
skip_count=0
while continue_flag==1:
    query_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit={limit}&skip={skip_count}'
    print(query_url)
    skip_count = skip_count + limit
    r = requests.get(query_url)
    j = json.loads(r.content)
    print(len(j['results']))
    if len(j['results'])!=1000:
        continue_flag==0

https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000&skip=0
1000
https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000&skip=1000


KeyError: 'results'

It seem, skip clause can't be used with count queries
https://opendata.stackexchange.com/questions/2163/how-to-get-total-count-of-adverse-effects-events-by-manufacturer-in-openfda

In [47]:
def get_data(query_url):
    r = requests.get(query_url)
    j = json.loads(r.content)
    return(pd.DataFrame(j['results']))

In [50]:
male_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
female_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:1+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
male_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:2+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
female_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:2+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'

In [51]:
df_male_ser = get_data(male_serious_url)
df_female_ser = get_data(female_serious_url)
df_male_nonser = get_data(male_nonserious_url)
df_female_nonser = get_data(female_nonserious_url)

In [54]:
df_male_ser.shape, df_female_ser.shape, df_male_nonser.shape, df_female_nonser.shape

((1000, 2), (482, 2), (48, 2), (17, 2))

In [58]:
def get_ae_pt_data(drug_brand_name):
    male_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    female_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:1+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    male_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:2+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    female_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:2+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2013-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    df_male_ser = get_data(male_serious_url)
    df_male_ser['gender']='male'
    df_male_ser['serious']=1
    df_female_ser = get_data(female_serious_url)
    df_female_ser['gender']='female'
    df_female_ser['serious']=1
    df_male_nonser = get_data(male_nonserious_url)
    df_male_nonser['gender']='male'
    df_male_nonser['serious']=0
    df_female_nonser = get_data(female_nonserious_url)
    df_female_nonser['gender']='female'
    df_female_nonser['serious']=0
    return(pd.concat([df_male_ser, df_female_ser,df_male_nonser,df_female_nonser], ignore_index=True))

In [59]:
tt = get_ae_pt_data('avastin')
tt.head()

,term,count,gender,serious
0,DEATH,420,male,1
1,DISEASE PROGRESSION,256,male,1
2,DIARRHOEA,106,male,1
3,NAUSEA,98,male,1
4,VOMITING,92,male,1


In [60]:
tt.tail()

,term,count,gender,serious
1542,RASH,1,female,0
1543,SPINE MALFORMATION,1,female,0
1544,THROMBOCYTOPENIA,1,female,0
1545,VISION BLURRED,1,female,0
1546,VISUAL IMPAIRMENT,1,female,0
